In [1]:
import numpy as np

import matplotlib.pyplot as plt
import scipy

# %matpnlotlib notebook 
%matplotlib inline 

# steady state channel considered here!

In [2]:
import qutip as qutip

# https://qutip.readthedocs.io/en/latest/guide/guide.html
# https://qutip.org/qutip-tutorials/

fileroot = 'SS_00_'

In [3]:
# check to see if an np.array square matrix is unitary 
def is_unitary(m):
    bb = np.allclose(np.eye(len(m)), m.dot(m.T.conj()), rtol=1e-05, atol=1e-08) # for square numpy array m
    return bb  # returns boolean
    
# given a Hermitian operator qutip.Qobj Hop on a 2 state system
# create an tensor product n_qubit unitary that is identity except j, j+1 qubits
# were we put the operator e^(i epsilon Hop)
# does not use any globals
def mk_interaction_small(n_qubits,Hop,epsilon,j):
    if (j <0):
        print('err j<0')
        return 0
    if (j > n_qubits-2):
        print('err j<n_qubits-2')
        return 0
    if (Hop.check_herm() != True):
        print('mk_interaction_small: Warning: Hop is not Hermitian')

    U_small_np = scipy.linalg.expm(1j*epsilon*Hop.full())  # make a small near identity transformation 
    # using scipy's expm with the hope that it will more likely to be actually unitary, the result is a numpy array 
    U_small =  qutip.Qobj(U_small_np,dims = [[2, 2], [2, 2]])
    if (U_small.isunitary != True):
        print('mk_interaction_small: U_small is not unitary')
        
    # now we make embed the interaction in the full tensor product space 
    id2 = qutip.qeye(2)  # identity 
    op_list = []  # create a list of operators 
    for k in range(0,j):
        op_list = np.append(op_list,id2)
    op_list = np.append(op_list,U_small)
    for k in range(j+2,n_qubits):
         op_list = np.append(op_list,id2)
    U_int = qutip.tensor(op_list)  # tensor product 
    if (U_int.isunitary != True):
        print('mk_interaction_small: U_int is not unitary')
    return U_int


# make the two Kraus operators we need for a reset operation on a single qubit 
# returns 2 qutip.Qobj 
def mkKraus_reset():
    ket0 = qutip.basis(2, 0) # |0>
    ket1 = qutip.basis(2, 1) # |1>
    K0 = ket0*ket0.dag() #|0><0|
    K1 = ket0*ket1.dag() #|0><1|
    return K0,K1

# Make the 2 Kraus operators needed for a full n_qubit system 
# resturns 2 qutip.Qobj operators 
# but reseting only j-th qubit 
# uses above routine mkKraus_reset()
def mk_Kreset(n_qubits,j):
    K0,K1 = mkKraus_reset() # get the 2 1 bit kraus ops
    id2 = qutip.qeye(2)  # identity op
    op_list0 = []
    op_list1 = []
    for k in range(0,j):
        op_list0 = np.append(op_list0,id2)  # load up identities until j index 
        op_list1 = np.append(op_list1,id2)  
    op_list0 = np.append(op_list0,K0) 
    op_list1 = np.append(op_list1,K1) 
    for k in range(j+1,n_qubits):
        op_list0 = np.append(op_list0,id2)  # load up identities until last index 
        op_list1 = np.append(op_list1,id2)  
    K0_big = qutip.tensor(op_list0)
    K1_big = qutip.tensor(op_list1)
    return K0_big,K1_big


# do a reset on density operator rho on last qubit 
def reset_channel(rho,n_qubits):
    ilist = np.arange(0,n_qubits-1)  # keep all but last one 
    rho_short = rho.ptrace(ilist)    # partial trace out the last qubit 
    ket0 = qutip.basis(2, 0) # |0>
    K0 = ket0*ket0.dag() #|0><0|
    op_list = [ rho_short, K0] 
    rho_new =qutip.tensor(op_list) # make new_rho = (tr_n-1 rho) \otimes |0><0|
    return rho_new

#do a depolarization channel on first qubit 
def depol_channel(rho,n_qubits):
    ilist = np.arange(1,n_qubits) # keep all but first one
    rho_short = rho.ptrace(ilist)    # trace out first qubit 
    id2 = qutip.qeye(2)  # identity op
    op_list = [ id2/2,rho_short ] 
    rho_new =qutip.tensor(op_list)  # make new_rho = I/2 \otimes (tr_0 rho) 
    return rho_new

if (1==0):
    rho_new = reset_channel(junk.rho,junk.n_qubits)
    print(rho_new,junk.rho)
    rho_new = depol_channel(junk.rho,junk.n_qubits)
    print(rho_new)

In [85]:
# class definition for carrying out the steady state version of the stochastic random walk channel 
# depolarization op on first qubit
# reset on last qubit
# in between exponentially small versions of an operator that we pass  
# calls above routines mk_Kreset(), mkKraus_reset(), mk_interaction_small() 
# depol_channel(rho,n_qubits),  reset_channel(rho,n_qubits):
class walkit_SS():
    # init: arguments:
    #  n_qubits:  number of qubits
    #  epsilon:  strength of 2qubit local interaction  - can be an array  length n_qubits - 1
    #  H_2bits:  Hermitian 2qubit interaction operator qutip.Qobj
    #  sim_label:  a string label for simulation
    #  inter_type :   a string specifying type of interaction 
    def __init__(self,n_qubits,epsilon,H_2bits,sim_label='',inter_type='seq'):
        self.n_qubits = n_qubits  # number of qubits
        self.N = 2**n_qubits
        if (isinstance(epsilon, float)):
            self.epsilon_arr = epsilon + np.zeros(n_qubits-1)  # perturbation parameter or parameters 
        else:
            self.epsilon_arr = epsilon
            if (len(epsilon) < n_qubits - 1):
                print("epsilon array too short")
        self.H_2bits = H_2bits  # Hermitian op qutip.Qobj
        self.sim_label = sim_label  # a string label 
        self.inter_type = inter_type  # interaction type 

        # make Kraus operators for reset on last qubit 
        K0_big,K1_big = mk_Kreset(self.n_qubits,self.n_qubits-1)    # external routine 
        self.K0_big = K0_big # store the Kraus operators (n_qubit system)  qutip.Qobj
        self.K1_big = K1_big 

        # make Kraus operators for depolarization channel on first qubit ?
        
        self.init_rho()  # initialize to the ground state, makes self.rho
        self.mk_int_prod()  # make an interaction unitary  self.U_int_prod  using self.epsilon_arr and self.H_2bits

        # create density operator from ground state in all qubits 
        
    # creates self.rho, initialized to |0> all qubits 
    def init_rho(self):
        ket0 = qutip.basis(2, 0)
        op_list = []
        for j in range(self.n_qubits):
            op_list = np.append(op_list,ket0)
        psi  = qutip.tensor(op_list) # this is a ket, not an operator 
        self.rho = qutip.ket2dm(psi)  # make density operator from statevector psi
        self.rho_init = self.rho  # keep a copy

    # create a unitary operator that is a tensor product of identity on all qubits but the j-th one 
    # and is random Haar on the j-th one
    # indexing j=0 to n_qubits-1
    # does not use any globals
    # returned operater is a unitary on full n_qubit system
    def create_rand_op(self,j):
        id2 = qutip.qeye(2)  # identity on a single qubit 
        R2 = qutip.random_objects.rand_unitary(2)  # Haar random on 2 state  (single qubit)
        op_list = []  # create a list of operators  that is n_qubits long
        for k in range(0,j):
            op_list = np.append(op_list,id2)  # insert identities 
        op_list = np.append(op_list,R2)  # put in the random op
        for k in range(j+1,self.n_qubits):
            op_list = np.append(op_list,id2) # more identities 
        U_rand = qutip.tensor(op_list)   # tensor product of n_qubit number of 2 state systems 
        return U_rand

    # make an interaction unitary of a product of near identity operators based on a 2 qubit operator self.H_2bits
    # creates self.U_int_prod
    # this is a sequential product of 2qubit operators 
    def mk_int_prod(self):
        if (self.inter_type == 'seq'):  # we use a sequential product of interactions with strengths epsilon_arr
            U_int_prod  = mk_interaction_small(self.n_qubits,self.H_2bits,self.epsilon_arr[0],0) # external routine 
            # this is a n_qubit operator that has e^{i epsilon H_2bits} operating on first two qubits
            # and is identity in the rest 
            for j in range(1,self.n_qubits-1):  # from 1 to n-1
                U_int = mk_interaction_small(self.n_qubits,self.H_2bits,self.epsilon_arr[j],j)    # sequential local interactions
                U_int_prod = U_int*U_int_prod   # is a product of all these interactions operators 
                
            self.U_int_prod = U_int_prod  # returns nothing, but creates self.U_int_prod
            self.U_int_prod_dagger = U_int_prod.dag()

        if (self.inter_type == 'brick'): # even and odd bricks 
            U_int_prod_even = mk_interaction_small(self.n_qubits,self.H_2bits,self.epsilon_arr[0],0) # external routine 
            U_int_prod_odd  = mk_interaction_small(self.n_qubits,self.H_2bits,self.epsilon_arr[1],1) 
            for j in range(2,self.n_qubits-1,2):  # from 2 to n-1, in non-touching but proximate pairs
                U_int = mk_interaction_small(self.n_qubits,self.H_2bits,self.epsilon_arr[j],j)    # sequential local interactions
                U_int_prod_even = U_int*U_int_prod_even   # is a product of all these interactions operators, pairs
            for j in range(3,self.n_qubits-1,2):  # from 3 to n-1, in non-touching but proximate pairs
                U_int = mk_interaction_small(self.n_qubits,self.H_2bits,self.epsilon_arr[j],j)    # sequential local interactions
                U_int_prod_odd = U_int*U_int_prod_odd   # is a product of all these interactions operators, pairs
            U_int_prod = U_int_prod_odd*U_int_prod_even

            self.U_int_prod = U_int_prod  # returns nothing, but creates self.U_int_prod
            
        if (self.U_int_prod.isunitary != True):
            print('warning: U_int_prod is not unitary!')

        bb = is_unitary(self.U_int_prod.full())
        if (bb != True):
            print('warning: U_int_prod is not unitary (bb)')

    # first do depol, then interactions, then reset, then update rho
    def do_SS_channel(self,do_checks=False):
        # first carry out depolarization channel 
        new_rho = depol_channel(self.rho,self.n_qubits)
        new_rho2 = self.U_int_prod*(new_rho*self.U_int_prod_dagger)  # unitary evolution of density operator 
        new_rho3 = reset_channel(new_rho2,self.n_qubits)
        self.rho = new_rho3  # update density operator 
        #print(np.trace(self.rho.full()))

    # iterate ntimes x nstore
    # store traced density operators nstore times 
    def iterate(self,ntimes,nstore,do_checks=False):
        rho_store = np.zeros([nstore,self.n_qubits,2,2],dtype=complex)
        mx_store = np.zeros([nstore,self.n_qubits])
        my_store = np.zeros([nstore,self.n_qubits])
        mz_store = np.zeros([nstore,self.n_qubits])
        for k in range(nstore):
            for m in range(ntimes):  # iterate ntimes 
                self.do_SS_channel(do_checks=do_checks)
            for j in range(self.n_qubits):
                rho_j = self.rho.ptrace(j)  # partial trace all qubits but the jth one
                rho_store[k,j,:,:] = rho_j.full()  # try to store each partial trace 
                mx = np.real( (rho_j*qutip.sigmax()).tr())  # measure x,y,z in each qubit
                my = np.real( (rho_j*qutip.sigmay()).tr())  #
                mz = np.real( (rho_j*qutip.sigmaz()).tr())
                mx_store[k,j] = mx
                my_store[k,j] = my
                mz_store[k,j] = mz
            if (do_checks==True):
                rhot = np.trace(self.rho.full())
                if (rhot > 1.2):
                    print('warning: trace rho >1.2')  # if you have screwed up badly!
                    break
        self.rho_store = rho_store
        self.mx_store = mx_store
        self.my_store = my_store
        self.mz_store = mz_store
        self.ntimes = ntimes
        self.nstore = nstore
        #self.print_lasts()
        print(self.mx_store[-1,:])
        print(self.my_store[-1,:])
        print(self.mz_store[-1,:])

    # compute expectation values of current density op
    def compute_exps(self):
        mx_arr = np.zeros(self.n_qubits)
        my_arr = np.zeros(self.n_qubits)
        mz_arr = np.zeros(self.n_qubits)
        for j in range(self.n_qubits):
            rho_j = self.rho.ptrace(j)  
            mx = np.real( (rho_j*qutip.sigmax()).tr())  # measure x,y,z in each qubit
            my = np.real( (rho_j*qutip.sigmay()).tr())  #
            mz = np.real( (rho_j*qutip.sigmaz()).tr())
            mx_arr[j] = mx
            my_arr[j] = my
            mz_arr[j] = mz
        return mx_arr,my_arr,mz_arr 

    # iterate untill reach a precission level for
    # the square root of the sum of squares of differences of expectation values of sigma_z in each qubit
    # using two iterations 
    def iterate_tol(self,ntimes_max,tol,do_checks=False):
        mz_old = np.ones(self.n_qubits)  + 3
        diff = 1
        nit = 0
        while (diff > tol):
            self.do_SS_channel(do_checks=do_checks) # iterate channel
            self.do_SS_channel(do_checks=do_checks) # iterate channel
            # compute current moments
            mx_arr,my_arr,mz_arr  = self.compute_exps()
            diff = np.sqrt(np.sum((mz_arr-mz_old)**2))
            # compute the square root of the sum of squares of differences of expectation values of sigma_z in each qubit
            mz_old = mz_arr 
            nit += 1
            self.mx_arr = mx_arr
            self.my_arr = my_arr
            self.mz_arr = mz_arr
            #print(mz_arr,diff)
            if (nit>ntimes_max):  # you have exceeded max number of iterations 
                print(mx_arr)
                print(my_arr)
                print(mz_arr)
                return 0,diff
        print(mx_arr)
        print(my_arr)
        print(mz_arr)
        return nit,diff  # return the number of iterations and the difference 
            
    def print_lasts(self):
        print(self.mx_store[-1,:])
        print(self.my_store[-1,:])
        print(self.mz_store[-1,:])

In [123]:
QHeis = qutip.Qobj([[1, 0, 0, 0], [0, -1, 2, 0], [0, 2, -1, 0], [0, 0, 0, 1]],
            dims = [[2, 2], [2, 2]], isherm=True)
epsilon = 0.05; n_qubits = 3; simlabel = ''
t1 = walkit_SS(n_qubits,epsilon,QHeis) # ,sim_label='',inter_type='seq'
#junk.do_SS_channel()
#ntimes = 1000; nstore = 10
#t1.iterate(ntimes,nstore,do_checks=True)
#t1.print_lasts()
ntimes_max = 2000; tol= 1e-5
nit,diff = t1.iterate_tol(ntimes_max,tol)
print(nit,diff)


[0. 0. 0.]
[0. 0. 0.]
[0.00501072 0.50274033 1.        ]
191 9.655435291147472e-06


In [124]:
epsilon = 0.02; n_qubits = 4; simlabel = ''
ss_a0= walkit_SS(n_qubits,epsilon,QHeis) # ,sim_label='',inter_type='seq'
ntimes_max = 5000; tol=1e-8
nit,diff = ss_a0.iterate_tol(ntimes_max,tol)
print(nit,diff)


[0. 0. 0. 0.]
[0. 0. 0. 0.]
[6.85665794e-04 4.28769746e-01 5.71920317e-01 1.00000000e+00]
3853 9.976971513109506e-09


In [32]:
print(3./7, 4./7)

0.42857142857142855 0.5714285714285714


In [91]:
epsilon = 0.01; n_qubits = 5; simlabel = ''
ss_b0= walkit_SS(n_qubits,epsilon,QHeis) # ,sim_label='',inter_type='seq'
ntimes_max = 10000; tol=1e-8
nit,diff = ss_b0.iterate_tol(ntimes_max,tol)
print(nit,diff)


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[1.58400100e-04 3.96052200e-01 5.07051243e-01 6.10384858e-01
 1.00000000e+00]
0 2.9275761120889002e-06


In [122]:

print(9./21)
print(7./15)

print(6./13)
print(4./9)
print(7./16)
print(11./26)
print(3./7)
print(5./12)
print(7./17)
print(2.0/5)

print(5./13,'5/13')

print(3./8)
print(4./11)
print(5./14)
print(6./17)


0.42857142857142855
0.4666666666666667
0.46153846153846156
0.4444444444444444
0.4375
0.4230769230769231
0.42857142857142855
0.4166666666666667
0.4117647058823529
0.4
0.38461538461538464 5/13
0.375
0.36363636363636365
0.35714285714285715
0.35294117647058826


In [176]:
def mk_Pauli_op(klist):
    op_list = []
    for j in range(len(klist)):
        k = klist[j]
        #print(k)
        if (k == 0):
            op = qutip.qeye(2)  # 0 gives identity 
        elif (k==1):
            op = qutip.sigmax() # 1 gives X 
        elif (k==2):
            op = qutip.sigmay()  # 2 gives Y
        else :
            op = qutip.sigmaz()  # 3 gives Z
            
        op_list = np.append(op_list,op)   # t list of Paulis
        
    Pauli_op = qutip.tensor(op_list)  # tensor product  of Paulis
    return Pauli_op
            

# compute expection value of a product of single qubit Paulis 
def compute_exps(rho,klist):
    # klist is a list of indices in 0,1,2,3 where 0 is identity, 1 is pauli X etc 
    Pauli_op =  mk_Pauli_op(klist)
    Prho = rho*Pauli_op
    expval = np.real(Prho.tr()) 
    return expval 

if (1==0): #test
    klist = np.array([0, 1, 3, 0, 0] )
    #mk_Pauli_op(klist)
    expv = compute_exps(ss_b0.rho,klist)
    print(expv)

def singles(sim):
    for j in range(1,4):
        for m in range(0,sim.n_qubits):
            klist = np.zeros(sim.n_qubits, dtype = int)
            klist[m] = j
            expv = compute_exps(sim.rho,klist)
            print(klist,expv)

def all_pairs_sames(sim,sepa):
    for j in range(1,4):
        #klist = np.zeros(sim.n_qubits, dtype = int)
        for m in range(0,sim.n_qubits):
            klist = np.zeros(sim.n_qubits, dtype = int)
            klist[m] = j
            klist[(m+sepa)%sim.n_qubits] = j
            expv = compute_exps(sim.rho,klist)
            print(klist,expv)

def pairs_difs(sim,sepa):
    set_index = np.array([1,2,3],dtype = int)
    for j in range(3):
        #klist = np.zeros(sim.n_qubits, dtype = int)
        for m in range(0,sim.n_qubits):
            klist = np.zeros(sim.n_qubits, dtype = int)
            klist[m] = set_index[j] 
            klist[(m+sepa)%sim.n_qubits] = set_index[(j+1)%3]
            expv = compute_exps(sim.rho,klist)
            print(klist, expv)

            klist = np.zeros(sim.n_qubits, dtype = int)
            klist[(m+sepa)%sim.n_qubits] = set_index[j] 
            klist[m] =  set_index[(j+1)%3]
            expv = compute_exps(sim.rho,klist)
            print(klist ,expv)

def all_sames(sim):
    for j in range(4):
        klist = np.zeros(sim.n_qubits, dtype = int) + j
        expv = compute_exps(sim.rho,klist)
        print(klist, expv)

all_sames(ss_b0)
print('------')
singles(ss_b0)
print('------')
all_pairs_sames(ss_b0,1)
print('------')
all_pairs_sames(ss_b0,2)
print('------')
pairs_difs(ss_b0,1)
print('------')
pairs_difs(ss_b0,2)


[0 0 0 0 0] 0.999999999998649
[1 1 1 1 1] 0.0
[2 2 2 2 2] 0.0
[3 3 3 3 3] 2.9045154302520526e-11
------
[1 0 0 0 0] 0.0
[0 1 0 0 0] 0.0
[0 0 1 0 0] 0.0
[0 0 0 1 0] 0.0
[0 0 0 0 1] 0.0
[2 0 0 0 0] 0.0
[0 2 0 0 0] 0.0
[0 0 2 0 0] 0.0
[0 0 0 2 0] 0.0
[0 0 0 0 2] 0.0
[3 0 0 0 0] 0.00015840010015036408
[0 3 0 0 0] 0.3960521995228002
[0 0 3 0 0] 0.5070512431053144
[0 0 0 3 0] 0.6103848579650432
[0 0 0 0 3] 0.999999999998649
------
[1 1 0 0 0] 0.000148274744314897
[0 1 1 0 0] 0.13222430084806272
[0 0 1 1 0] 0.13843576163637847
[0 0 0 1 1] 0.0
[1 0 0 0 1] 0.0
[2 2 0 0 0] 0.000148274744314897
[0 2 2 0 0] 0.13222430084806272
[0 0 2 2 0] 0.13843576163637847
[0 0 0 2 2] 0.0
[2 0 0 0 2] 0.0
[3 3 0 0 0] 0.00010576119857408275
[0 3 3 0 0] 0.238606716247729
[0 0 3 3 0] 0.2613543035717167
[0 0 0 3 3] 0.6103848579650432
[3 0 0 0 3] 0.00015840010015036408
------
[1 0 1 0 0] -9.531274765067474e-05
[0 1 0 1 0] 0.07580110233792198
[0 0 1 0 1] 0.0
[1 0 0 1 0] 3.021803741307629e-05
[0 1 0 0 1] 0.0
[2 0 2 0 0]

 The terms in the density operator that are non zero are 
1,2,0,0,0 and permutations 
3,3,0,0,0 
2,2,0,0,0
1,1,0,0,0
3,0,0,0,0

Terms that are zero are 
2,0,0,0,0 and permutations 
1,0,0,0,0
2,3,0,0,0
1,3,0,0,0
all 1 
all 2 
all 3 

I haven't yet checked the triples 